In [11]:
#!/usr/bin/env python3
"""DQN agents trained on Breakthrough by independent Q-learning."""
from utils.environment.game import GraphGame
from utils.environment.nodeCentrality import Node_Centrality
from utils.environment.globalFeature import Global_Feature
from utils.reinforcement_learning.rl_environment import Environment
from utils.validation import *
from utils.evaluation import jsonEncoder
from utils.evaluation.evaluationhelper import *
from utils.environment.envhelper import gen_new_graphs
from utils.reinforcement_learning.dqn_TS import DQN
from utils.hyperparameters.params import Params
from utils.getClass import objective_function,get_class_from_file
from tqdm import tqdm
import torch
import numpy as np
import wandb
import os
import matplotlib.pyplot as plt
class BenchMark():
    def __init__(self,path):
        #path: relative path to the parameter  
        self.params = Params(path)
        self.nodeCentrality = Node_Centrality(self.params.centrality_feature_name)
        self.globalFeature = Global_Feature(self.params.global_feature_name)
        self.objectiveFunction = objective_function("utils/environment/objectiveFunction.py",self.params.objective_function)
        self.GNN = get_class_from_file(self.params.GNN[0], self.params.GNN[1])
        os.environ['WANDB_NOTEBOOK_NAME'] = self.params.wandb_notebook_name
        wandb.login()
            
    def Generate_Batch_Graph(self, size,seed = None):
        Batch_Graph = [gen_new_graphs(graph_type=self.params.graph_type,seed=seed+i) for i in range(size)]
        return np.array(Batch_Graph,dtype=object)

    def validation(self,size,env, agent, evaluation, x):
        AUC = []
        for i in range(size):
            eval_step = env.reset(evaluation[i].copy(),self.objectiveFunction,self.nodeCentrality,self.globalFeature)
            while not eval_step.last():
                eval_output, prob = agent.step(eval_step, is_evaluation=True)
                eval_step = env.step(eval_output)
            gamma = env.get_state.reward1
            AUC.append(area_under_curve(x[i][:len(gamma)],gamma))
        meanAUC = np.mean(AUC)
        return meanAUC

    def bestModel(self, allEval):
        start = int(self.params.save_every)
        end = int(self.params.num_train_episodes)+start
        interval = int(self.params.save_every)
        model = np.arange(start, end, interval) 
        x = np.argmin(allEval)
        best = model[x]
        self.best_dir  = self.params.checkpoint_dir+"_"+best

    def input_graph(self, graph_path):
        GRAPH = Graph.Read_Ncol(graph_path, directed=False)
        nodes = [v.index for v in GRAPH.vs]
        map = {n:int(i) for i, n in enumerate(nodes)}
        GRAPH = reset(GRAPH)  
        Graph.simplify(GRAPH)
        return GRAPH, map

    def train(self,agent=None):
        game = GraphGame
        env = Environment(game)
        # WandB – Initialize a new run
        wandb.init(mode ="offline",entity="bhandk", project="Attack_and_Defense_Same_Graph_Type",name=self.params.wandb_name,config=self.params)
        wandb.watch_called = False # Re-run the model without restarting the runtime, unnecessary after our next release
        size_CV = self.params.validation_test_size
        evaluation, eval_x = get_Validation(size_CV,seed=0)#get_Validation(4,file_path)
        CV_AUC = []
        if agent == None:
            agent = DQN(
                    state_representation_size=self.params.centrality_features,
                    global_feature_size = self.params.global_features,
                    hidden_layers_sizes=self.params.hidden_layers,
                    replay_buffer_capacity=int(self.params.replay_buffer_capacity),
                    learning_rate=self.params.learning_rate,
                    update_target_network_every=  self.params.update_target_network_every,
                    learn_every=self.params.learn_every,
                    discount_factor=self.params.discount_factor,
                    min_buffer_size_to_learn=self.params.min_buffer_size_to_learn,
                    power = self.params.epsilon_power,
                    nsteps=self.params.nstep,
                    epsilon_start=self.params.epsilon_start,
                    epsilon_end=self.params.epsilon_end,
                    epsilon_decay_duration=self.params.epsilon_decay_duration,
                    batch_size=self.params.batch_size,
                    GraphNN = self.GNN)
        #agents.append(random_agent.RandomAgent(player_id=1, num_actions=num_actions))
        wandb.watch(agent._q_network, log="all")
        graph_batch_size = self.params.graph_batch_size
        for ep in tqdm(range(int(self.params.num_train_episodes))):
            if (ep) % self.params.graph_suffle == 0:
                Batch_Graph = self.Generate_Batch_Graph(graph_batch_size,seed=ep)
            time_step = env.reset(Batch_Graph[int(ep%graph_batch_size)].copy(),self.objectiveFunction,self.nodeCentrality,self.globalFeature)
            while not time_step.last():
                action, prob = agent.step(time_step) 
                time_step = env.step(action)
            # Episode is over, step agent with final info state.
            agent.step(time_step)
            wandb.log({"loss": agent._last_loss_value,"cummulative_reward":env.get_state._returns,"epsilon":agent.epsilon})
            if (ep + 1) % self.params.eval_every == 0:
                meanAUC = self.validation(size_CV, env, agent, evaluation, eval_x)
                CV_AUC.append(meanAUC)
                wandb.log({"Evaluation_Dataset": meanAUC})          
            if (ep + 1) % self.params.save_every == 0:
                checkpoint = {'_q_network': agent._q_network.state_dict(),'target_q_network': agent._target_q_network.state_dict(),'_optimizer' :agent._optimizer.state_dict()}
                title = self.params.checkpoint_dir+"_"+str(ep+1)
                torch.save(checkpoint, title)
        wandb.finish() 
        return self.bestModel(CV_AUC)

    def evaluation(self, graphpath, bestModel=None, useSingleStep= None, useIndex = False):
        """
        Given a path to an edgelist of the Graph and a model, it evaluates the removal of node based on the objective function. 
        """
        if self.objectiveFunction.__name__ == "numberConnectedComponent":
            condMaxNum = True
        else:
            condMaxNum = False
        G , _ = self.input_graph(graphpath)
        if bestModel == None:
            bestModel = self.best_dir 
        model = torch.load(self.params.checkpoint_dir+"_"+bestModel)
        N = G.vcount()
        game = GraphGame
        env = Environment(game)
        attacker = DQN(state_representation_size=self.params.centrality_features,
                            hidden_layers_sizes= self.params.hidden_layers,
                           global_feature_size = self.params.global_features)
        attacker._q_network.load_state_dict(model["_q_network"])
        attacker._optimizer.load_state_dict(model["_optimizer"])
        rewards, value, actions = EvaluateModel(env, self.objectiveFunction, self.nodeCentrality, self.globalFeature, attacker, G,useSingleStep)
        x =  np.flip(np.arange(N)[N:0:-1]/N)
        auc = area_under_curve(condMaxNum,N,x[:len(value)],value)
        fraction = len(actions)/N
        return auc, fraction
    
    def evaluationOthers(self, graphpath, actionspath, useIndex =False):
        """
        Given a path to an edgelist of a Graph, the list of action sequences, "actions" in order, and an objectiveFunction it evaluates the removal of node based on the objective function. 
        """
        import warnings
        if self.objectiveFunction.__name__ == "numberConnectedComponent":
            condMaxNum = True
        else:
            condMaxNum = False
        G , _ = self.input_graph(graphpath)
        N = G.vcount()
        fname = actionspath+graphpath.split("/")[-1]
        if os.path.exists(fname):
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                actions = np.loadtxt(fname, dtype=int, unpack=True)
                if len(np.shape(actions)) == 0:
                    actions = [actions.item()]
            if len(actions)!= 0 :
                rewards, value, actions = EvaluateACTION(actions,self.objectiveFunction,G, useIndex)
                if condMaxNum:
                    x =  np.flip(np.arange(N+1)[N:0:-1]/N)
                else:
                    x =  np.flip(np.arange(N+1)[N:0:-1]/N)
                value = np.array(value)/value[0]
                auc = area_under_curve(condMaxNum,N,x[:len(value)],value)
                fraction = len(actions)/N
                return auc, fraction
            else:
                return None, None
        else:
            return None,None

    def compareBenchmarkSynthetic(self,evaluation,bestmodel, useIndex = False):
        """
        Evaluate Graph on Synthetic Graph. 
        """
        '''syntheticGraph = {"Degree":"./Dataset/SyntheticGraph/Degree/New/",
                    "Homogeneity":"./Dataset/SyntheticGraph/Homogeneity/New/", 
                   }
        result = {"Degree": { "barabasi" : {"graphname": [], "auc": [], "fraction": []},
                           "erdos" : {"graphname": [], "auc": [], "fraction": []},
                           "small-world" : {"graphname": [], "auc": [], "fraction": []},
                           "regular" : {"graphname": [], "auc": [], "fraction": []},
                           "sbm" : {"graphname": [], "auc": [], "fraction": []}
                             
                           },
                  "Homogeneity": { "barabasi" : {"graphname": [], "auc": [], "fraction": []},
                           "erdos" : {"graphname": [], "auc": [], "fraction": []},
                           "small-world" : {"graphname": [],  "auc": [], "fraction": []}
                           }
                 }
        for folder in syntheticGraph:
            graphlist = os.listdir(syntheticGraph[folder]) 
            if ".ipynb_checkpoints" in graphlist:
                graphlist.remove(".ipynb_checkpoints")
            #graphlist = [g for g in graphlist if int(g.split("_")[-1].split(".")[0])<6]
            for graphname in graphlist:
                name = graphname.split("_")[0]
                graphpath = syntheticGraph[folder] + graphname
                value = evaluation(graphpath,bestmodel,useIndex = useIndex)
                if value[0] != None:
                    result[folder][name]["graphname"].append(graphname)
                    result[folder][name]["auc"].append(value[0])
                    result[folder][name]["fraction"].append(value[1])
        self.result_Syn = result'''
        syntheticGraph = {"synthetic":"./Dataset/SyntheticGraph/New/"}
        result = {"synthetic": { "barabasi" : {"graphname": [], "auc": [], "fraction": []},
                           "erdos" : {"graphname": [], "auc": [], "fraction": []},
                           "small-world" : {"graphname": [], "auc": [], "fraction": []},
                           "regular" : {"graphname": [], "auc": [], "fraction": []},
                           "sbm" : {"graphname": [], "auc": [], "fraction": []}
                             
                           }
                 }
        for folder in syntheticGraph:
            graphlist = os.listdir(syntheticGraph[folder]) 
            if ".ipynb_checkpoints" in graphlist:
                graphlist.remove(".ipynb_checkpoints")
            #graphlist = [g for g in graphlist if int(g.split("_")[-1].split(".")[0])<6]
            for graphname in graphlist:
                name = graphname.split("_")[0]
                graphpath = syntheticGraph[folder] + graphname
                value = evaluation(graphpath,bestmodel,useIndex = useIndex)
                if value[0] != None:
                    result[folder][name]["graphname"].append(graphname)
                    result[folder][name]["auc"].append(value[0])
                    result[folder][name]["fraction"].append(value[1])
        self.result_Syn = result
        
    def compareBenchmarkSyntheticMotif(self,evaluation,bestmodel, useIndex = False):
        """
        Evaluate Graph on Synthetic Graph with Motifs attached. 
        """
        syntheticMotifGraph = {"BA":"./Dataset/Motifs_Attached/New_BA/",
                    "Tree":"./Dataset/Motifs_Attached/New_Tree/", 
                   }
        result = {"BA": { "cycle" : {"graphname": [],"auc": [], "fraction": []},
                                  "clique" : {"graphname": [], "auc": [], "fraction": []},
                                  "house" : {"graphname": [], "auc": [], "fraction": []},
                                  "grid" : {"graphname": [], "auc": [], "fraction": []},
                                  "star" : {"graphname": [], "auc": [], "fraction": []},
                                  "fan" : {"graphname": [], "auc": [], "fraction": []},
                                  "diamond" : {"graphname": [], "auc": [], "fraction": []}
                                },
                 "Tree": { "cycle" : {"graphname": [], "auc": [], "fraction": []},
                                  "clique" : {"graphname": [], "auc": [], "fraction": []},
                                  "house" : {"graphname": [], "auc": [], "fraction": []},
                                  "grid" : {"graphname": [], "auc": [], "fraction": []},
                                  "star" : {"graphname": [], "auc": [], "fraction": []},
                                  "fan" : {"graphname": [], "auc": [], "fraction": []},
                                  "diamond" : {"graphname": [], "auc": [], "fraction": []}
                                }
                 }
        for folder in syntheticMotifGraph:
            graphlist = os.listdir(syntheticMotifGraph[folder])
            #graphlist.remove(".ipynb_checkpoints")
            for graphname in graphlist:
                name = graphname.split("_")[3]
                graphpath = syntheticMotifGraph[folder] + graphname
                if os.path.exists(graphpath):
                    value = evaluation(graphpath,bestmodel,useIndex = useIndex)
                    if value[0] != None:
                        result[folder][name]["graphname"].append(graphname)
                        result[folder][name]["auc"].append(value[0])
                        result[folder][name]["fraction"].append(value[1])
        self.result_SynMotif = result

In [12]:
import json
import sys
import json
from utils.evaluation.jsonEncoder import NpEncoder
def get_from_json(file_path):
    with open(file_path) as json_file:
        data = json.load(json_file)
    return data
SAVE = get_from_json("./Figure/BoxPlots/Result.json")
SAVE = get_from_json("./Figure/BoxPlots/ResultNew1.json")

# Largest Connected Components

In [ ]:
#Best
#ba = BenchMark("./utils/hyperparameters/BA/ba_params_Best.json")
ba = BenchMark("./utils/hyperparameters/BA/ba_params_Best_New.json")
print("Best: Synthetic")
#ba.compareBenchmarkSynthetic(ba.evaluation,"204500")#166000
ba.compareBenchmarkSynthetic(ba.evaluation,"66500")#166000 #RE EVALUATED- inf
#print("Best: SyntheticMotif")
#ba.compareBenchmarkSyntheticMotif(ba.evaluation,"204500")#166000

#MIX
mix = BenchMark("./utils/hyperparameters/Mix/mixed_params_All.json")
print("Mix: Synthetic")
mix.compareBenchmarkSynthetic(mix.evaluation,"85000")
#print("Mix: SyntheticMotif")
#mix.compareBenchmarkSyntheticMotif(mix.evaluation,"85000")
#FINDER
finder = BenchMark("./utils/hyperparameters/Mix/mixed_params_All.json")
print("Finder: Synthetic")
finder.compareBenchmarkSynthetic(finder.evaluationOthers,"./FINDER/FINDER_ReTrained/", useIndex=True)
#print("Finder: SyntheticMotif")
#finder.compareBenchmarkSyntheticMotif(finder.evaluationOthers,"./FINDER/FINDER_ReTrained/")

#CI
ci = BenchMark("./utils/hyperparameters/Mix/mixed_params_All.json")
print("CI: Synthetic")
ci.compareBenchmarkSynthetic(ci.evaluationOthers,"./CI/L_2/")
#print("CI: SyntheticMotif")
#ci.compareBenchmarkSyntheticMotif(ci.evaluationOthers,"./CI/L_2/")

#GDM
gdm = BenchMark("./utils/hyperparameters/Mix/mixed_params_All.json")
print("GDM: Synthetic")
gdm.compareBenchmarkSynthetic(gdm.evaluationOthers,"./GDM/",useIndex=True)
#print("GDM: SyntheticMotif")
#gdm.compareBenchmarkSyntheticMotif(gdm.evaluationOthers,"./GDM/",useIndex=True)

#CoreHD
core = BenchMark("./utils/hyperparameters/Mix/mixed_params_All.json")
'''print("Core: Synthetic")
core.compareBenchmarkSynthetic(core.evaluationOthers,"./CoreHD/")
print("Core: SyntheticMotif")
core.compareBenchmarkSyntheticMotif(core.evaluationOthers,"./CoreHD/")'''
print("Core: Synthetic")
core.compareBenchmarkSynthetic(core.evaluationOthers,"./CoreHD/COREHD_New/")
#print("Core: SyntheticMotif")
#core.compareBenchmarkSyntheticMotif(core.evaluationOthers,"./CoreHD/COREHD_New/")

obj = [ba, mix, finder, ci,gdm,core]
#obj = [ba_CN]
model_name =  ['ba', 'mix', 'finder', 'ci','gdm','corehd']
model_name =  ['ba', 'mix', 'finder', 'ci','gdm','corehd_new']
for i,models in enumerate(obj):
    result = {}
    #for method in ["result_Syn","result_SynMotif"]:
    for method in ["result_Syn"]:
        result_method = getattr(models, method) 
        result[method] = result_method
    SAVE[model_name[i]] = result


In [36]:
#Best
ba_new = BenchMark("./utils/hyperparameters/BA/ba_params_Best_New.json")
print("Best: Synthetic")
#ba_new.compareBenchmarkSynthetic(ba_new.evaluation,"225500")#166000
#ba_new.compareBenchmarkSynthetic(ba_new.evaluation,"241000")#166000 #RE EVALUATED 
#ba_new.compareBenchmarkSynthetic(ba_new.evaluation,"298500")#166000 #RE EVALUATED- foodweb
#ba_new.compareBenchmarkSynthetic(ba_new.evaluation,"66500")#166000 #RE EVALUATED- inf
ba_new.compareBenchmarkSynthetic(ba_new.evaluation,"120000")#166000 #RE EVALUATED- corr

#MIX
mix_new= BenchMark("./utils/hyperparameters/Mix/mixed_params_Best_New.json")
print("Mix: Synthetic")
#mix_new.compareBenchmarkSynthetic(mix_new.evaluation,"279000") #RE EVALUATED- 
#mix_new.compareBenchmarkSynthetic(mix_new.evaluation,"87500") #RE EVALUATED- foodweb
#mix_new.compareBenchmarkSynthetic(mix_new.evaluation,"89500") #RE EVALUATED- inf
mix_new.compareBenchmarkSynthetic(mix_new.evaluation,"177500") #RE EVALUATED- corr
obj = [ba_new, mix_new]
#obj = [ba_CN]
model_name =  ['ba', 'mix', 'finder', 'ci','gdm','corehd']
model_name =  ['ba_last', 'mix_last']
for i,models in enumerate(obj):
    result = {}
    #for method in ["result_Syn","result_SynMotif"]:
    for method in ["result_Syn"]:
        result_method = getattr(models, method) 
        result[method] = result_method
    SAVE[model_name[i]] = result


Best: Synthetic
Mix: Synthetic


# Pairwise Connectivity

In [86]:
#ba_CN = BenchMark("./utils/hyperparameters/BA/ba_params_CN.json")
ba_CN = BenchMark("./utils/hyperparameters/BA/ba_params_CN_New.json")
print("BA: Synthetic")
ba_CN.compareBenchmarkSynthetic(ba_CN.evaluation,"52000") # CN_NEW
#ba_CN.compareBenchmarkSynthetic(ba_CN.evaluation,"137000") # CN
#print("BA: SyntheticMotif")
#ba_CN.compareBenchmarkSyntheticMotif(ba_CN.evaluation,"137000")

#MIX
#mix_CN = BenchMark("./utils/hyperparameters/Mix/mixed_params_CN.json")
mix_CN = BenchMark("./utils/hyperparameters/Mix/mixed_params_CN_New.json")
print("Mix: Synthetic")
mix_CN.compareBenchmarkSynthetic(mix_CN.evaluation,"56000")# CN_NEW
#mix_CN.compareBenchmarkSynthetic(mix_CN.evaluation,"98500") CN
#print("Mix: SyntheticMotif")
#mix_CN.compareBenchmarkSyntheticMotif(mix_CN.evaluation,"98500")

#FINDER
finder_CN = BenchMark("./utils/hyperparameters/Mix/mixed_params_CN.json")
print("Finder: Synthetic")
finder_CN.compareBenchmarkSynthetic(finder_CN.evaluationOthers,"./FINDER/FINDER_CN/",useIndex=True)
#print("Finder: SyntheticMotif")
#finder_CN.compareBenchmarkSyntheticMotif(finder_CN.evaluationOthers,"./FINDER/FINDER_CN/")
#CI
ci_CN = BenchMark("./utils/hyperparameters/Mix/mixed_params_CN.json")
print("CI: Synthetic")
ci_CN.compareBenchmarkSynthetic(ci_CN.evaluationOthers,"./CI/L_2/")
#print("CI: SyntheticMotif")
#ci_CN.compareBenchmarkSyntheticMotif(ci_CN.evaluationOthers,"./CI/L_2/")

#GDM
gdm_CN = BenchMark("./utils/hyperparameters/Mix/mixed_params_CN.json")
print("GDM: Synthetic")
gdm_CN.compareBenchmarkSynthetic(gdm_CN.evaluationOthers,"./GDM/GDM_CN/",useIndex=True)
#print("GDM: SyntheticMotif")
#gdm_CN.compareBenchmarkSyntheticMotif(gdm_CN.evaluationOthers,"./GDM/",useIndex=True)

#CoreHD
core_CN = BenchMark("./utils/hyperparameters/Mix/mixed_params_CN.json")
'''print("Core: Synthetic")
core_CN.compareBenchmarkSynthetic(core_CN.evaluationOthers,"./CoreHD/")
print("Core: SyntheticMotif")
core_CN.compareBenchmarkSyntheticMotif(core_CN.evaluationOthers,"./CoreHD/")'''
print("Core: Synthetic")
core_CN.compareBenchmarkSynthetic(core_CN.evaluationOthers,"./CoreHD/COREHD_New/")
#print("Core: SyntheticMotif")
#core_CN.compareBenchmarkSyntheticMotif(core_CN.evaluationOthers,"./CoreHD/COREHD_New/")

obj = [ba_CN, mix_CN, finder_CN, ci_CN,gdm_CN,core_CN]
model_name = ['ba_CN', 'mix_CN', 'finder_CN', 'ci_CN','gdm_CN','corehd_CN']
model_name = ['ba_CN', 'mix_CN', 'finder_CN', 'ci_CN','gdm_CN','corehd_new_CN']
for i,models in enumerate(obj):
    result = {}
    #for method in ["result_Syn","result_SynMotif"]:
    for method in ["result_Syn"]:
        result_method = getattr(models, method) 
        result[method] = result_method
    SAVE[model_name[i]] = result

BA: Synthetic
Mix: Synthetic
Finder: Synthetic
CI: Synthetic
GDM: Synthetic
Core: Synthetic


In [13]:
#FINDER
finder_CN = BenchMark("./utils/hyperparameters/Mix/mixed_params_CN.json")
print("Finder: Synthetic")
finder_CN.compareBenchmarkSynthetic(finder_CN.evaluationOthers,"./FINDER/FINDER_CN/",useIndex=True)


gdm_CN = BenchMark("./utils/hyperparameters/Mix/mixed_params_CN.json")
print("GDM: Synthetic")
gdm_CN.compareBenchmarkSynthetic(gdm_CN.evaluationOthers,"./GDM/GDM_CN/",useIndex=True)
#print("GDM: SyntheticMotif")
#gdm_CN.compareBenchmarkSyntheticMotif(gdm_CN.evaluationOthers,"./GDM/",useIndex=True)


obj = [finder_CN,gdm_CN]
model_name = ['finder_CN','gdm_CN']
for i,models in enumerate(obj):
    result = {}
    #for method in ["result_Syn","result_SynMotif"]:
    for method in ["result_Syn"]:
        result_method = getattr(models, method) 
        result[method] = result_method
    SAVE[model_name[i]] = result

Finder: Synthetic
GDM: Synthetic


## MaxNum: Maximum Number of Connected Components

In [119]:
ba_MaxNum = BenchMark("./utils/hyperparameters/BA/ba_params_MaxNum.json")
print("BA: Synthetic")
ba_MaxNum.compareBenchmarkSynthetic(ba_MaxNum.evaluation,"258500")
#print("BA: SyntheticMotif")
#ba_MaxNum.compareBenchmarkSyntheticMotif(ba_MaxNum.evaluation,"258500")

#MIX
mix_MaxNum = BenchMark("./utils/hyperparameters/Mix/mixed_params_MaxNum.json")
print("Mix: Synthetic")
mix_MaxNum.compareBenchmarkSynthetic(mix_MaxNum.evaluation,"284000")
#print("Mix: SyntheticMotif")
#mix_MaxNum.compareBenchmarkSyntheticMotif(mix_MaxNum.evaluation,"284000")
#CI
ci_MaxNum = BenchMark("./utils/hyperparameters/Mix/mixed_params_MaxNum.json")
print("CI: Synthetic")
ci_MaxNum.compareBenchmarkSynthetic(ci_MaxNum.evaluationOthers,"./CI/L_2/")
#print("CI: SyntheticMotif")
#ci_MaxNum.compareBenchmarkSyntheticMotif(ci_MaxNum.evaluationOthers,"./CI/L_2/")

#CoreHD
core_MaxNum = BenchMark("./utils/hyperparameters/Mix/mixed_params_MaxNum.json")
'''print("Core: Synthetic")
core_MaxNum.compareBenchmarkSynthetic(core_MaxNum.evaluationOthers,"./CoreHD/")
print("Core: SyntheticMotif")
core_MaxNum.compareBenchmarkSyntheticMotif(core_MaxNum.evaluationOthers,"./CoreHD/")'''
print("Core: Synthetic")
core_MaxNum.compareBenchmarkSynthetic(core_MaxNum.evaluationOthers,"./CoreHD/COREHD_New/")
#print("Core: SyntheticMotif")
#core_MaxNum.compareBenchmarkSyntheticMotif(core_MaxNum.evaluationOthers,"./CoreHD/COREHD_New/")


obj = [ba_MaxNum, mix_MaxNum, ci_MaxNum,core_MaxNum]
model_name = ['ba_MaxNum', 'mix_MaxNum',  'ci_MaxNum','corehd_MaxNum']
model_name = ['ba_MaxNum', 'mix_MaxNum',  'ci_MaxNum','corehd_new_MaxNum']
for i,models in enumerate(obj):
    result = {}
    #for method in ["result_Syn","result_SynMotif"]:
    for method in ["result_Syn"]:
        result_method = getattr(models, method) 
        result[method] = result_method
    SAVE[model_name[i]] = result

BA: Synthetic
Mix: Synthetic
CI: Synthetic
Core: Synthetic


In [14]:
#FINDER
finder_MaxNum = BenchMark("./utils/hyperparameters/Mix/mixed_params_MaxNum.json")
print("Finder: Synthetic")
finder_MaxNum.compareBenchmarkSynthetic(finder_MaxNum.evaluationOthers,"./FINDER/FINDER_MaxNum/",useIndex=True)
#FINDER
GDM_MaxNum = BenchMark("./utils/hyperparameters/Mix/mixed_params_MaxNum.json")
print("Finder: Synthetic")
GDM_MaxNum.compareBenchmarkSynthetic(GDM_MaxNum.evaluationOthers,"./GDM/GDM_MaxNum/",useIndex=True)




obj = [finder_MaxNum,GDM_MaxNum]
model_name = ['finder_MaxNum','gdm_MaxNum']
for i,models in enumerate(obj):
    result = {}
    #for method in ["result_Syn","result_SynMotif"]:
    for method in ["result_Syn"]:
        result_method = getattr(models, method) 
        result[method] = result_method
    SAVE[model_name[i]] = result

Finder: Synthetic
Finder: Synthetic


# Comparison with Global Feature

## BA Training Graph

In [ ]:
#Best
ba_all = BenchMark("./utils/hyperparameters/BA/ba_params_All.json")
print("Best: Synthetic")
ba_all.compareBenchmarkSynthetic(ba_all.evaluation,"400500")
print("Best: SyntheticMotif")
ba_all.compareBenchmarkSyntheticMotif(ba_all.evaluation,"400500")

#Density
den = BenchMark("./utils/hyperparameters/BA/ba_params_Density.json")
print("density: Synthetic")
den.compareBenchmarkSynthetic(den.evaluation,"471000")
print("density: SyntheticMotif")
den.compareBenchmarkSyntheticMotif(den.evaluation,"471000")

#Entropy
entropy = BenchMark("./utils/hyperparameters/BA/ba_params_Entropy.json")
print("entropy: Synthetic")
entropy.compareBenchmarkSynthetic(entropy.evaluation,"132000")
print("entropy: SyntheticMotif")
entropy.compareBenchmarkSyntheticMotif(entropy.evaluation,"132000")

#Hetero
hetero = BenchMark("./utils/hyperparameters/BA/ba_params_Heterogeneity.json")
print("Hetero: Synthetic")
hetero.compareBenchmarkSynthetic(hetero.evaluation,"103500")
print("Hetero: SyntheticMotif")
hetero.compareBenchmarkSyntheticMotif(hetero.evaluation,"103500")

#Gini
gini = BenchMark("./utils/hyperparameters/BA/ba_params_Gini.json")
print("gini: Synthetic")
gini.compareBenchmarkSynthetic(gini.evaluation,"250500")
print("gini: SyntheticMotif")
gini.compareBenchmarkSyntheticMotif(gini.evaluation,"250500")

#Resilience
resilience = BenchMark("./utils/hyperparameters/BA/ba_params_Resilience.json")
print("resilience: Synthetic")
resilience.compareBenchmarkSynthetic(resilience.evaluation,"497000")
print("resilience: SyntheticMotif")
resilience.compareBenchmarkSyntheticMotif(resilience.evaluation,"497000")


#Transitivity
transi = BenchMark("./utils/hyperparameters/BA/ba_params_Transitivity.json")
print("Transi: Synthetic")
transi.compareBenchmarkSynthetic(transi.evaluation,"457000")
print("Transi: SyntheticMotif")
transi.compareBenchmarkSyntheticMotif(transi.evaluation,"457000")

#None
none = BenchMark("./utils/hyperparameters/BA/ba_params_None.json")
print("None: Synthetic")
none.compareBenchmarkSynthetic(none.evaluation,"405000")
print("None: SyntheticMotif")
none.compareBenchmarkSyntheticMotif(none.evaluation,"405000")


obj = [ba_all, den, entropy, hetero, gini, resilience, transi, none]
model_name = ['All_ba', 'Density_ba', 'Entropy_ba', 'Heterogeneity_ba','Gini_ba','Resilience_ba', 'Transitivity_ba', 'None_ba']
for i,models in enumerate(obj):
    result = {}
    for method in ["result_Syn","result_SynMotif"]:
        result_method = getattr(models, method) 
        result[method] = result_method
    SAVE[model_name[i]] = result

## Mixed Training Graph

In [ ]:

#Density
den_mix = BenchMark("./utils/hyperparameters/Mix/mixed_params_Density.json")
print("density: Synthetic")
den_mix.compareBenchmarkSynthetic(den_mix.evaluation,"43000")
print("density: SyntheticMotif")
den_mix.compareBenchmarkSyntheticMotif(den_mix.evaluation,"43000")

#Entropy
entropy_mix = BenchMark("./utils/hyperparameters/Mix/mixed_params_Entropy.json")
print("entropy: Synthetic")
entropy_mix.compareBenchmarkSynthetic(entropy_mix.evaluation,"119500")
print("entropy: SyntheticMotif")
entropy_mix.compareBenchmarkSyntheticMotif(entropy_mix.evaluation,"119500")

#Hetero
hetero_mix = BenchMark("./utils/hyperparameters/Mix/mixed_params_Heterogeneity.json")
print("Hetero: Synthetic")
hetero_mix.compareBenchmarkSynthetic(hetero_mix.evaluation,"108500")
print("Hetero: SyntheticMotif")
hetero_mix.compareBenchmarkSyntheticMotif(hetero_mix.evaluation,"108500")

#gini
gini_mix = BenchMark("./utils/hyperparameters/Mix/mixed_params_Gini.json")
print("gini: Synthetic")
gini_mix.compareBenchmarkSynthetic(gini_mix.evaluation,"80000")
print("gini: SyntheticMotif")
gini_mix.compareBenchmarkSyntheticMotif(gini_mix.evaluation,"80000")

#Resilience
resilience_mix = BenchMark("./utils/hyperparameters/Mix/mixed_params_Resilience.json")
print("resilience: Synthetic")
resilience_mix.compareBenchmarkSynthetic(resilience_mix.evaluation,"32000")
print("resilience: SyntheticMotif")
resilience_mix.compareBenchmarkSyntheticMotif(resilience_mix.evaluation,"32000")

#Transitivity
transi_mix = BenchMark("./utils/hyperparameters/Mix/mixed_params_Transitivity.json")
print("Transi: Synthetic")
transi_mix.compareBenchmarkSynthetic(transi_mix.evaluation,"29000")
print("Transi: SyntheticMotif")
transi_mix.compareBenchmarkSyntheticMotif(transi_mix.evaluation,"29000")

#None
none_mix = BenchMark("./utils/hyperparameters/Mix/mixed_params_None.json")
print("None: Synthetic")
none_mix.compareBenchmarkSynthetic(none_mix.evaluation,"206500")
print("None: SyntheticMotif")
none_mix.compareBenchmarkSyntheticMotif(none_mix.evaluation,"206500")



obj = [den_mix, entropy_mix, hetero_mix, gini_mix, resilience_mix, transi_mix, none_mix]
model_name = ['Density_mix', 'Entropy_mix', 'Heterogeneity_mix','Gini_mix','Resilience_mix', 'Transitivity_mix', 'None_mix']
for i,models in enumerate(obj):
    result = {}
    for method in ["result_Syn","result_SynMotif"]:
        result_method = getattr(models, method) 
        result[method] = result_method
    SAVE[model_name[i]] = result

## Small World Training Graph

In [ ]:
#Best
sw = BenchMark("./utils/hyperparameters/SW/sw_params_All.json")
print("All: Synthetic")
sw.compareBenchmarkSynthetic(sw.evaluation,"193000")
print("All: SyntheticMotif")
sw.compareBenchmarkSyntheticMotif(sw.evaluation,"193000") 

#Density
den_sw = BenchMark("./utils/hyperparameters/SW/sw_params_Density.json")
print("density: Synthetic")
den_sw.compareBenchmarkSynthetic(den_sw.evaluation,"254500")
print("density: SyntheticMotif")
den_sw.compareBenchmarkSyntheticMotif(den_sw.evaluation,"254500")

#Entropy
entropy_sw = BenchMark("./utils/hyperparameters/SW/sw_params_Entropy.json")
print("entropy: Synthetic")
entropy_sw.compareBenchmarkSynthetic(entropy_sw.evaluation,"58500")
print("entropy: SyntheticMotif")
entropy_sw.compareBenchmarkSyntheticMotif(entropy_sw.evaluation,"58500")

#Hetero
hetero_sw = BenchMark("./utils/hyperparameters/SW/sw_params_Heterogeneity.json")
print("Hetero: Synthetic")
hetero_sw.compareBenchmarkSynthetic(hetero_sw.evaluation,"346000")
print("Hetero: SyntheticMotif")
hetero_sw.compareBenchmarkSyntheticMotif(hetero_sw.evaluation,"346000")

#gini
gini_sw = BenchMark("./utils/hyperparameters/SW/sw_params_Gini.json")
print("gini: Synthetic")
gini_sw.compareBenchmarkSynthetic(gini_sw.evaluation,"462000")
print("gini: SyntheticMotif")
gini_sw.compareBenchmarkSyntheticMotif(gini_sw.evaluation,"462000")

#Resilience
resilience_sw = BenchMark("./utils/hyperparameters/SW/sw_params_Resilience.json")
print("resilience: Synthetic")
resilience_sw.compareBenchmarkSynthetic(resilience_sw.evaluation,"157000")
print("resilience: SyntheticMotif")
resilience_sw.compareBenchmarkSyntheticMotif(resilience_sw.evaluation,"157000")

#Transitivity
transi_sw = BenchMark("./utils/hyperparameters/SW/sw_params_Transitivity.json")
print("Transi: Synthetic")
transi_sw.compareBenchmarkSynthetic(transi_sw.evaluation,"243000")
print("Transi: SyntheticMotif")
transi_sw.compareBenchmarkSyntheticMotif(transi_sw.evaluation,"243000")

#None
none_sw = BenchMark("./utils/hyperparameters/SW/sw_params_None.json")
print("None: Synthetic")
none_sw.compareBenchmarkSynthetic(none_sw.evaluation,"39500")
print("None: SyntheticMotif")
none_sw.compareBenchmarkSyntheticMotif(none_sw.evaluation,"39500")



obj = [sw, den_sw, entropy_sw, hetero_sw, gini_sw, resilience_sw, transi_sw, none_sw]
model_name = ['sw', 'Density_sw', 'Entropy_sw', 'Heterogeneity_sw','Gini_sw','Resilience_sw', 'Transitivity_sw', 'None_sw']
for i,models in enumerate(obj):
    result = {}
    for method in ["result_Syn","result_SynMotif"]:
        result_method = getattr(models, method) 
        result[method] = result_method
    SAVE[model_name[i]] = result

## Erdos Renyi Training Graph

In [ ]:
#Best
er = BenchMark("./utils/hyperparameters/ER/er_params_All.json")
print("All: Synthetic")
er.compareBenchmarkSynthetic(er.evaluation,"27000")
print("All: SyntheticMotif")
er.compareBenchmarkSyntheticMotif(er.evaluation,"27000") 

#Density
den_er = BenchMark("./utils/hyperparameters/ER/er_params_Density.json")
print("density: Synthetic")
den_er.compareBenchmarkSynthetic(den_er.evaluation,"48500")
print("density: SyntheticMotif")
den_er.compareBenchmarkSyntheticMotif(den_er.evaluation,"48500")

#Entropy
entropy_er = BenchMark("./utils/hyperparameters/ER/er_params_Entropy.json")
print("entropy: Synthetic")
entropy_er.compareBenchmarkSynthetic(entropy_er.evaluation,"48000")
print("entropy: SyntheticMotif")
entropy_er.compareBenchmarkSyntheticMotif(entropy_er.evaluation,"48000")

#Hetero
hetero_er = BenchMark("./utils/hyperparameters/ER/er_params_Heterogeneity.json")
print("Hetero: Synthetic")
hetero_er.compareBenchmarkSynthetic(hetero_er.evaluation,"22500")
print("Hetero: SyntheticMotif")
hetero_er.compareBenchmarkSyntheticMotif(hetero_er.evaluation,"22500")

#gini
gini_er = BenchMark("./utils/hyperparameters/ER/er_params_Gini.json")
print("gini: Synthetic")
gini_er.compareBenchmarkSynthetic(gini_er.evaluation,"16000")
print("gini: SyntheticMotif")
gini_er.compareBenchmarkSyntheticMotif(gini_er.evaluation,"16000")

#Resilience
resilience_er = BenchMark("./utils/hyperparameters/ER/er_params_Resilience.json")
print("resilience: Synthetic")
resilience_er.compareBenchmarkSynthetic(resilience_er.evaluation,"126500")
print("resilience: SyntheticMotif")
resilience_er.compareBenchmarkSyntheticMotif(resilience_er.evaluation,"126500")

#Transitivity
transi_er = BenchMark("./utils/hyperparameters/ER/er_params_Transitivity.json")
print("Transi: Synthetic")
transi_er.compareBenchmarkSynthetic(transi_er.evaluation,"38000")
print("Transi: SyntheticMotif")
transi_er.compareBenchmarkSyntheticMotif(transi_er.evaluation,"38000")

#None
none_er = BenchMark("./utils/hyperparameters/ER/er_params_None.json")
print("None: Synthetic")
none_er.compareBenchmarkSynthetic(none_er.evaluation,"144500")
print("None: SyntheticMotif")
none_er.compareBenchmarkSyntheticMotif(none_er.evaluation,"144500")



obj = [er, den_er, entropy_er, hetero_er, gini_er, resilience_er, transi_er, none_er]
model_name = ['er', 'Density_er', 'Entropy_er', 'Heterogeneity_er','Gini_er','Resilience_er', 'Transitivity_er', 'None_er']
for i,models in enumerate(obj):
    result = {}
    for method in ["result_Syn","result_SynMotif"]:
        result_method = getattr(models, method) 
        result[method] = result_method
    SAVE[model_name[i]] = result

# SAVE to JSON file

In [16]:
with open("./Figure/BoxPlots/ResultNew1.json", "w") as outfile:
    json.dump(SAVE, outfile, cls=NpEncoder)